In [1]:
import trimesh
from tqdm import tqdm
import numpy as np
import json
import pdb

In [2]:
csv_dir = "/home/akita/cs182/stl.json"
with open(csv_dir, 'r') as r:
    mesh_dict = json.load(r)


In [3]:
below_five_counter = 0
selected = []
for mesh in tqdm(mesh_dict):
    if len(selected) > 5000:
        break
    m = trimesh.load(mesh)
    if m.is_watertight:
        label = mesh_dict[mesh]
        if float(label) < 5:
            below_five_counter += 1
            if below_five_counter > 2500:
                continue
        selected.append(mesh)

 56%|█████▋    | 31163/55266 [04:54<03:47, 105.81it/s] 


In [8]:
sanitized_dict = {}
for mesh in tqdm(selected):
    m = trimesh.load(mesh)
    tsfm_matrix = np.eye(4)
    tsfm_matrix[:3, 3] = -m.center_mass
    m = m.apply_transform(tsfm_matrix)
    trimesh.exchange.export.export_mesh(
        m, mesh.replace("rotated_files", "repaired_files")
    )
    sanitized_dict["repaired_files/" + mesh.split('/')[-1]] = mesh_dict[mesh]

100%|██████████| 5001/5001 [01:23<00:00, 59.98it/s] 


In [9]:
with open("sanitized_dict.json", 'w') as w:
    json.dump(sanitized_dict, w)